In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import json
from datetime import datetime
from sibal17_train import NavigationEnv
from sibal17_train import DQNAgent

# CUDA 디바이스 설정 (사용자 코드와 동일)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 테스트 함수 정의
def test_dqn(env, agent, model_path, num_episodes=10):
    """
    학습된 DQN 모델을 테스트하는 함수
    
    Parameters:
    - env: NavigationEnv 객체
    - agent: DQNAgent 객체
    - model_path: 학습된 모델 가중치 파일 경로
    - num_episodes: 테스트할 에피소드 수
    """
    # 모델 불러오기
    agent.policy_net.load_state_dict(torch.load(model_path))
    agent.policy_net.eval()  # 평가 모드로 전환 (dropout 등 비활성화)
    
    # 결과 저장 디렉토리 설정
    base_dir = r"C:\baramproject\trained_model\sibal17"
    image_dir = os.path.join(base_dir, "test_images")
    data_dir = os.path.join(base_dir, "test_data")
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(data_dir, exist_ok=True)
    
    # 테스트 에피소드 실행
    for episode in range(num_episodes):
        state = env.reset()  # 환경 초기화
        total_reward = 0
        path_length = 0
        done = False
        debug_data = []  # 디버깅 데이터 저장용 리스트
        
        # 에피소드 실행
        while not done:
            # 탐색 없이 행동 선택 (epsilon=0)
            action = agent.select_action(state, epsilon=0)
            next_state, reward, done, _ = env.step(action)
            
            # Q-값 계산 및 기록
            q_values = agent.policy_net(torch.FloatTensor(state).unsqueeze(0).to(device)).detach().cpu().numpy().flatten()
            
            # 디버깅 데이터 추가
            debug_data.append({
                "step": path_length,
                "state": state.tolist(),
                "action": action,
                "reward": reward,
                "next_state": next_state.tolist(),
                "q_values": q_values.tolist()
            })
            
            # 상태 업데이트
            state = next_state
            total_reward += reward
            path_length += 1
        
        # 결과 출력
        print(f"Test Episode {episode}, Total Reward: {total_reward:.2f}, Path Length: {path_length}")
        
        # 경로 시각화
        plt.figure(figsize=(10, 8))
        plt.imshow(env.grid, cmap='gray')  # 환경 그리드 표시
        path_array = np.array(env.path)
        plt.plot(path_array[:, 1], path_array[:, 0], 'r-', label='Path')  # 경로 플롯
        plt.plot(env.start_pos[1], env.start_pos[0], 'go', label='Start')  # 시작점
        plt.plot(env.end_pos[1], env.end_pos[0], 'bo', label='End')  # 종료점
        plt.legend()
        plt.title(f"Test Episode {episode} Path")
        plt.savefig(os.path.join(image_dir, f"test_episode_{episode}.png"))
        plt.close()
        
        # 디버깅 데이터 저장
        with open(os.path.join(data_dir, f"test_episode_{episode}.json"), 'w') as f:
            json.dump(debug_data, f, indent=4)

# 메인 실행
if __name__ == "__main__":
    # 환경 및 에이전트 초기화 (사용자 코드와 동일)
    env = NavigationEnv()
    state_dim = 7  # 상태 차원
    action_dim = len(env.action_space)  # 행동 차원
    agent = DQNAgent(state_dim, action_dim)
    
    # 학습된 모델 경로
    model_path = r"C:\baramproject\trained_model\sibal17\navigation_model.pth"
    
    # 테스트 실행
    test_dqn(env, agent, model_path, num_episodes=10)